# Interactive Visualization Lab

Complete the following set of exercises to solidify your knowledge of interactive visualization using Plotly, Cufflinks, and IPyWidgets.

In [33]:
import pandas as pd
import chart_studio.plotly as py
import cufflinks as cf
from ipywidgets import interact
import plotly.express as px

#cf.go_offline()

In [34]:
data = pd.read_excel('../data/Online Retail.xlsx')

In [35]:

# data_canada = px.data.gapminder().query("country == 'Canada'")
# fig = px.bar(data_canada, x='year', y='pop')
# fig.show()

data.head()

,InvoiceNo,InvoiceDate,StockCode,Description,Quantity,UnitPrice,Revenue,CustomerID,Country
0,536365,2010-12-01 08:26:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,6,2.55,15.3,17850,United Kingdom
1,536373,2010-12-01 09:02:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,6,2.55,15.3,17850,United Kingdom
2,536375,2010-12-01 09:32:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,6,2.55,15.3,17850,United Kingdom
3,536390,2010-12-01 10:19:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,64,2.55,163.2,17511,United Kingdom
4,536394,2010-12-01 10:39:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,32,2.55,81.6,13408,United Kingdom


## 1. Create an interactive bar chart showing total quantity and revenue by country (excluding United Kingdom) for the month of April 2011.

In [36]:
brexit = data[(data['Country'] != 'United Kingdom') & (data['InvoiceDate'].astype(str).str.contains('^20.{2}-04-*', regex = True))]
brexit.head()

,InvoiceNo,InvoiceDate,StockCode,Description,Quantity,UnitPrice,Revenue,CustomerID,Country
724,549667,2011-04-11 12:20:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,6,2.95,17.70,14911,EIRE
807,551163,2011-04-26 15:52:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,9,2.95,26.55,12573,France
3320,550899,2011-04-21 12:07:00,22752,SET 7 BABUSHKA NESTING BOXES,2,8.50,17.00,13505,Switzerland
4880,550527,2011-04-19 10:48:00,84879,ASSORTED COLOUR BIRD ORNAMENT,40,1.69,67.60,12476,Germany
4882,550620,2011-04-19 13:39:00,84879,ASSORTED COLOUR BIRD ORNAMENT,32,1.69,54.08,12585,Germany


In [37]:
brexitCountry = brexit.groupby('Country').agg({'Quantity':'sum', 'Revenue': 'sum'})

In [38]:
brexitCountry.iplot(kind='bar', filename='cufflinks/categorical-bar-chart')


## 2. Create an interactive line chart showing quantity and revenue sold to France between January 1st and May 31st 2011.

In [39]:
frenchbags = data[(data['Country'] == 'France') & (data['InvoiceDate'] >= '01-01-2011') & (data['InvoiceDate'] <= '31-05-2011')]
frenchbags.head()

,InvoiceNo,InvoiceDate,StockCode,Description,Quantity,UnitPrice,Revenue,CustomerID,Country
807,551163,2011-04-26 15:52:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,9,2.95,26.55,12573,France
2645,540976,2011-01-12 15:00:00,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,4,3.75,15.00,12652,France
3290,545181,2011-02-28 14:45:00,22752,SET 7 BABUSHKA NESTING BOXES,4,8.50,34.00,12509,France
4614,542629,2011-01-31 09:57:00,84879,ASSORTED COLOUR BIRD ORNAMENT,160,1.45,232.00,12731,France
4795,548409,2011-03-31 10:27:00,84879,ASSORTED COLOUR BIRD ORNAMENT,160,1.45,232.00,12731,France


In [40]:
frenchgrouped = frenchbags.groupby('InvoiceDate').agg({'Quantity':'sum', 'Revenue': 'sum'})
frenchgrouped.head()
fig = frenchgrouped.iplot(filename='cufflinks/line-example')

## 3. Create an interactive scatter plot showing the relationship between average quantity (x-axis) and average unit price (y-axis) for the product PARTY BUNTING with the plot points color-coded by country (categories).

In [41]:
bunting = data[data['Description'] == 'PARTY BUNTING']
bunting.head()

,InvoiceNo,InvoiceDate,StockCode,Description,Quantity,UnitPrice,Revenue,CustomerID,Country
252864,536956,2010-12-03 12:43:00,47566,PARTY BUNTING,5,4.65,23.25,14210,United Kingdom
252865,537065,2010-12-05 11:57:00,47566,PARTY BUNTING,5,4.65,23.25,12567,France
252866,537128,2010-12-05 12:15:00,47566,PARTY BUNTING,2,4.65,9.30,12841,United Kingdom
252867,537142,2010-12-05 12:57:00,47566,PARTY BUNTING,1,4.65,4.65,12748,United Kingdom
252868,537420,2010-12-06 15:18:00,47566,PARTY BUNTING,5,4.65,23.25,17519,United Kingdom


In [42]:
buntingbyCountry = bunting.groupby('Country').agg({'Quantity':'mean', 'UnitPrice': 'mean'})
buntingbyCountry.head()

,Quantity,UnitPrice
Country,,
Australia,33.125000,4.7125
Austria,8.000000,4.9500
Belgium,4.000000,4.9500
Channel Islands,13.333333,4.9500
Cyprus,2.333333,4.7500


In [43]:
buntfig = buntingbyCountry.iplot(kind='scatter', 
                                 xTitle='Quantity', 
                                 yTitle='Unit Price', 
                                 mode='markers', 
                                 x='Quantity', 
                                 y='UnitPrice', 
                                 title='Mean quantity vs. mean Unitprice, by Country', 
                                 filename='cufflinks/simple-scatter')
# buntfig.update_layout(title_text='Your Mom') Esto ultimo no me va, no se porque

In [44]:
buntingbyCountry.reset_index(inplace=True)

In [45]:

fig = px.scatter(buntingbyCountry, x='Quantity', y="UnitPrice", color="Country",
                 )
fig.show()

## 4. Create a set of interactive histograms showing the distributions of quantity per invoice for the following countries: EIRE, Germany, France, and Netherlands.

In [46]:
fewcountries = data[data['Country'].isin(['EIRE', 'Germany', 'France', 'Netherlands'])]

In [47]:
fewcountries.head()

,InvoiceNo,InvoiceDate,StockCode,Description,Quantity,UnitPrice,Revenue,CustomerID,Country
179,539320,2010-12-16 19:16:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,24,2.95,70.8,14911,EIRE
198,539722,2010-12-21 13:45:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,24,2.95,70.8,14911,EIRE
304,541570,2011-01-19 12:34:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,256,2.55,652.8,14646,Netherlands
322,541979,2011-01-24 14:54:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,6,2.95,17.7,14911,EIRE
367,542777,2011-02-01 08:31:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,6,2.95,17.7,14911,EIRE


In [48]:

quantityperinvoice = fewcountries.pivot_table(values='Quantity', columns='Country', 
                      index='InvoiceNo', aggfunc='sum')
quantityperinvoice.head()

Country,EIRE,France,Germany,Netherlands
InvoiceNo,,,,
536370,NaN,446.0,NaN,NaN
536403,NaN,NaN,NaN,96.0
536527,NaN,NaN,156.0,NaN
536540,230.0,NaN,NaN,NaN
536541,12.0,NaN,NaN,NaN


In [49]:
quantityperinvoice.iplot(kind='hist', xTitle='Quantity of Invoice', 
           yTitle='Amount of invoices', title='Distribution of invoice quantities')

## 5. Create an interactive side-by-side bar chart showing the revenue by country listed below (bars) for each of the products listed below.

In [50]:
product_list = ['JUMBO BAG RED RETROSPOT', 
                'CREAM HANGING HEART T-LIGHT HOLDER',
                'REGENCY CAKESTAND 3 TIER']

country_list = ['EIRE', 'Germany', 'France', 'Netherlands']

In [51]:
select = data[(data['Country'].isin(country_list)) & (data['Description'].isin(product_list))]
select.head()

,InvoiceNo,InvoiceDate,StockCode,Description,Quantity,UnitPrice,Revenue,CustomerID,Country
179,539320,2010-12-16 19:16:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,24,2.95,70.8,14911,EIRE
198,539722,2010-12-21 13:45:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,24,2.95,70.8,14911,EIRE
304,541570,2011-01-19 12:34:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,256,2.55,652.8,14646,Netherlands
322,541979,2011-01-24 14:54:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,6,2.95,17.7,14911,EIRE
367,542777,2011-02-01 08:31:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,6,2.95,17.7,14911,EIRE


In [52]:
revbyProd = select.pivot_table(values='Revenue', 
                               columns='Country', 
                               index='Description', 
                               aggfunc='sum')
revbyProd

Country,EIRE,France,Germany,Netherlands
Description,,,,
CREAM HANGING HEART T-LIGHT HOLDER,2740.80,131.75,35.40,1167.00
JUMBO BAG RED RETROSPOT,278.72,903.37,1072.76,3468.00
REGENCY CAKESTAND 3 TIER,7388.55,2816.85,9061.95,3166.35


In [53]:
revbyProd.iplot(kind='bar', 
                xTitle='Product', 
                yTitle='Revenue', 
                title='Revenue per product, by country', 
                filename='cufflinks/categorical-bar-chart')

## 6. Create an interactive line chart showing quantity sold by day for the United Kingdom. Add drop-down boxes for Year and Month that allow you to filter the date range that appears in the chart.

In [54]:
data['Year'] = pd.DatetimeIndex(data['InvoiceDate']).year
data['Month'] = pd.DatetimeIndex(data['InvoiceDate']).month
data['Day'] = pd.DatetimeIndex(data['InvoiceDate']).day
uk = data[data['Country']=='United Kingdom']

In [55]:
uk.head()

,InvoiceNo,InvoiceDate,StockCode,Description,Quantity,UnitPrice,Revenue,CustomerID,Country,Year,Month,Day
0,536365,2010-12-01 08:26:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,6,2.55,15.3,17850,United Kingdom,2010,12,1
1,536373,2010-12-01 09:02:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,6,2.55,15.3,17850,United Kingdom,2010,12,1
2,536375,2010-12-01 09:32:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,6,2.55,15.3,17850,United Kingdom,2010,12,1
3,536390,2010-12-01 10:19:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,64,2.55,163.2,17511,United Kingdom,2010,12,1
4,536394,2010-12-01 10:39:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,32,2.55,81.6,13408,United Kingdom,2010,12,1


In [56]:
ukpivot = uk.pivot_table(values='Quantity', 
               columns=['Month','Year'], 
               index='Day', 
               aggfunc='sum').reset_index()
ukpivot.head()

Month Day        1        2        3        4        5        6        7  \
Year          2011     2011     2011     2011     2011     2011     2011   
0       1      NaN  13536.0   7614.0  17149.0   3172.0   6091.0   6078.0   
1       2      NaN   8184.0   8148.0      NaN      NaN  12587.0      NaN   
2       3      NaN  14502.0   7842.0   5667.0   9566.0   6429.0   3117.0   
3       4   6659.0  10226.0  11186.0  10937.0  15624.0      NaN   9438.0   
4       5  17635.0      NaN      NaN  10564.0  13906.0  10412.0  17063.0   

Month        8        9       10       11       12           
Year      2011     2011     2011     2011     2010     2011  
0      10484.0  22003.0      NaN  14648.0  21308.0  23502.0  
1      10371.0  13732.0   8375.0  22681.0  30987.0  22960.0  
2      15124.0      NaN  24420.0  30681.0   7646.0      NaN  
3      35474.0   7516.0  22678.0  31205.0      NaN   9855.0  
4       9982.0  16021.0  32641.0      NaN  13603.0  35874.0

In [57]:
ukpivot.iplot(kind='line', x='Day')

In [58]:
@interact(Selection=['Year', 'Month'])

def linechart(Selection):
    dats = uk.pivot_table(values='Quantity', columns=Selection,
                            index='Day', aggfunc='mean').reset_index()

    dats.iplot(kind='line', x='Day')

# Unfortunately the dropdown menu does not show up...

In [61]:
@interact(Month=list(uk['Month'].unique()), 
          Year=list(uk['Year'].unique())
         )

def liner(Month, Year):
    data6 = uk[(uk['Month']==Month) & 
              (uk['Year']==Year)]
    data6.drop(columns=['InvoiceNo','StockCode','Description','UnitPrice','Revenue','CustomerID','Country','Year','Month','Day'], inplace=True)
    data6.sort_values('InvoiceDate', inplace=True)
    data6.iplot(kind='line', x='InvoiceDate', xTitle='Time', color='Red',
           yTitle='Quantity', title='Quantity in UK by time')

## 7. Create an interactive scatter plot that plots number of invoices (x-axis) vs. number of customers (y-axis) and the plot points represent individual products. Add two sliders that control the x and y axis ranges.

In [59]:
agg_func = {'InvoiceNo':'nunique',
            'Quantity':'sum',
            'UnitPrice':'mean',
            'Revenue':'sum',
            'CustomerID':'nunique'}

products = uk.groupby('Description').agg(agg_func)

## 8. Creat an interactive bar chart that shows revenue by product description. Add a text field widget that filters the results to show the product that contain the text entered in their description.